In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import random

In [ ]:
def scraper(ogurl, year,csv_save_url):
    import pandas as pd
    from bs4 import BeautifulSoup
    import requests

    stories_data = []

    for month in range(1, 13):
        if month in [1, 3, 5, 7, 8, 10, 12]:
            n_days = 31
        elif month in [4, 6, 9, 11]:
            n_days = 30
        else:
            n_days = 28

        for day in range(1, n_days + 1):

            month, day = str(month), str(day)

            if len(month) == 1:
                month = f'0{month}'
            if len(day) == 1:
                day = f'0{day}'

            date = f'{month}/{day}/{year}'
            url = f'{ogurl}/archive/{year}/{month}/{day}'
            print(url)
            page = requests.get(url)
            soup = BeautifulSoup(page.text, 'html.parser')

            stories = soup.find_all('div', class_='streamItem streamItem--postPreview js-streamItem')
            j=0
            for story in stories:
                j=0
                try:
                    each_story = []

                    author_box = story.find('div', class_='postMetaInline u-floatLeft u-sm-maxWidthFullWidth')
                    author_url = author_box.find('a')['href']

                    try:
                        reading_time = author_box.find('span', class_='readingTime')['title']
                    except:
                        continue

                    title = story.find('h3').text if story.find('h3') else '-'

                    subtitle = story.find('h4').text if story.find('h4') else '-'

                    if story.find('button', class_='button button--chromeless u-baseColor--buttonNormal'
                                               ' js-multirecommendCountButton u-disablePointerEvents'):

                        claps = story.find('button', class_='button button--chromeless u-baseColor--buttonNormal'
                                                        ' js-multirecommendCountButton u-disablePointerEvents').text
                    else:
                        claps = 0

                    if story.find('a', class_='button button--chromeless u-baseColor--buttonNormal'):
                        responses = story.find('a', class_='button button--chromeless u-baseColor--buttonNormal').text
                    else:
                        responses = '0 responses'

                    story_url = story.find('a', class_='button button--smaller button--chromeless u-baseColor--buttonNormal')[
                    'href']

                    reading_time = reading_time.split()[0]
                    responses = responses.split()[0]

                    story_page = requests.get(story_url)
                    story_soup = BeautifulSoup(story_page.text, 'html.parser')

                    sections = story_soup.find_all('section')
                    story_paragraphs = []
                    section_titles = []
                    for section in sections:
                        paragraphs = section.find_all('p')
                        for paragraph in paragraphs:
                            story_paragraphs.append(paragraph.text)

                        subs = section.find_all('h1')
                        for sub in subs:
                            section_titles.append(sub.text)

                    number_sections = len(section_titles)
                    number_paragraphs = len(story_paragraphs)

                    each_story.append(date)
                    each_story.append(title)
                    each_story.append(subtitle)
                    each_story.append(claps)
                    each_story.append(responses)
                    each_story.append(author_url)
                    each_story.append(story_url)
                    each_story.append(reading_time)
                    each_story.append(number_sections)
                    each_story.append(section_titles)
                    each_story.append(number_paragraphs)
                    each_story.append(story_paragraphs)

                    stories_data.append(each_story)
                except:
                    j+=1
                    continue


            print(f'{len(stories)-j} stories scraped on {date}.')

    columns = ['date', 'title', 'subtitle', 'claps', 'responses', 'author_url', 'story_url',
               'reading_time (mins)', 'number_sections', 'section_titles', 'number_paragraphs', 'paragraphs']

    df = pd.DataFrame(stories_data, columns=columns)
    df.to_csv(csv_save_url, sep='\t', index=False)
    return df

In [ ]:
swlh_df_2023 = scraper(r"https://medium.com/swlh/",2023,'swlh_df_2023.csv')
swlh_df_2022 = scraper(r"https://medium.com/swlh/",2022,'swlh_df_2022.csv')
swlh_df_2021 = scraper(r"https://medium.com/swlh/",2021,'swlh_df_2021.csv')

In [ ]:
towards_df_2023 = scraper(r"https://towardsdatascience.com/", 2023,'towards_df_2023.csv')
towards_df_2022 = scraper(r"https://towardsdatascience.com/", 2022,'towards_df_2022.csv')
towards_df_2021 = scraper(r"https://towardsdatascience.com/", 2021,'towards_df_2021.csv')


In [ ]:
import torch
from transformers import pipeline

def is_generated_by_ai(paragraph):
    # Load the text classification pipeline
    text_classifier = pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment")

    # Classify the input paragraph
    result = text_classifier(paragraph)

    # You can adjust this threshold based on experimentation
    confidence_threshold = 0.7

    # Check if the label is consistent with AI-generated text
    label = result[0]['label']
    confidence = result[0]['score']
    if label == 'LABEL_1' and confidence >= confidence_threshold:
        return True
    else:
        return False

# Example usage
input_paragraph = "This is an example paragraph."
generated_by_ai = is_generated_by_ai(input_paragraph)

if generated_by_ai:
    print("The paragraph seems to be generated by AI.")
else:
    print("The paragraph seems to be written by a human.")

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("Hello-SimpleAI/chatgpt-detector-roberta")
model = AutoModelForSequenceClassification.from_pretrained("Hello-SimpleAI/chatgpt-detector-roberta")

In [ ]:
from transformers import TextClassificationPipeline
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)
pipe("I have generated some mock data in Python that is obviously not linearly separable. However, there is a clear pattern separating the two classes which gives us an indication that we could create a function that splits the data in a higher dimension. So how about instead of trying to classify this data in two dimensions, we map the data to a three-dimensional space, and try to classify there.")

In [ ]:
pipe("Amelia sat on the porch, sipping her coffee, gazing at the old oak tree in her backyard. Its branches whispered tales of time, and today, they carried a peculiar message. As she listened, a small bird landed on her shoulder, its song harmonizing with the rustling leaves. Suddenly, a letter floated down from the tree, tied with a crimson ribbon. Intrigued, she opened it to find words penned by her late grandmother, a letter lost for decades. Through tears and laughter, the oak tree became the keeper of family stories, connecting generations in a dance of memories under the vast sky.")